# Author: ddukic

In [11]:
import sys
import evaluate

seqeval = evaluate.load("seqeval")

sys.path.append("../baselines/")

from dataset import (
    ACE2005TriggerRelationDataset,
    EDNYTTriggerRelationDataset,
    EVEXTRATriggerRelationDataset,
)

from transformers import AutoTokenizer

from util import build_vocab

all_labels_trigger, id2trigger, trigger2id = build_vocab(["Trigger"])
all_labels_relation, id2relation, relation2id = build_vocab(["Relation"])


tokenizer = AutoTokenizer.from_pretrained(
    "roberta-base", do_lower_case=False, add_prefix_space=True
)


def calculate_alignment(
    source_dataset, dataset_train_path, dataset_train_mini_extractions_path, tokenizer
):
    from torch.utils import data
    import torch

    dataset_test = source_dataset(
        fpath_trigger=dataset_train_path,
        fpath_relation=dataset_train_mini_extractions_path,
        tokenizer=tokenizer,
        trigger2id=trigger2id,
        relation2id=relation2id,
        implicit=True,
    )

    triggers = dataset_test.triggers
    relations = [
        [y.replace("Relation", "Trigger") for y in x] for x in dataset_test.relations
    ]

    to_del = []

    for i, (x, y) in enumerate(zip(triggers, relations)):
        if len(x) != len(y):
            to_del.append(i)

    for idx in to_del:
        del triggers[idx]
        del relations[idx]

    metrics = seqeval.compute(
        predictions=triggers,
        references=relations,
        scheme="IOB2",
        mode="strict",
    )

    print(metrics)

In [12]:
calculate_alignment(
    ACE2005TriggerRelationDataset,
    "../data/raw/ace/test.json",
    "../data/processed/mini/ace_test_triplets_filtered_merged.json",
    tokenizer,
)

{'Trigger': {'precision': 0.035545023696682464, 'recall': 0.03205128205128205, 'f1': 0.033707865168539325, 'number': 468}, 'overall_precision': 0.035545023696682464, 'overall_recall': 0.03205128205128205, 'overall_f1': 0.033707865168539325, 'overall_accuracy': 0.928023758099352}


In [13]:
calculate_alignment(
    EDNYTTriggerRelationDataset,
    "../data/processed/ednyt/test.json",
    "../data/processed/mini/ednyt_test_triplets_filtered_merged.json",
    tokenizer,
)

{'Trigger': {'precision': 0.021791767554479417, 'recall': 0.06382978723404255, 'f1': 0.03249097472924188, 'number': 141}, 'overall_precision': 0.021791767554479417, 'overall_recall': 0.06382978723404255, 'overall_f1': 0.03249097472924188, 'overall_accuracy': 0.9069833191970597}


In [14]:
calculate_alignment(
    EVEXTRATriggerRelationDataset,
    "../data/processed/evextra/test.json",
    "../data/processed/mini/evextra_test_triplets_filtered_merged.json",
    tokenizer,
)

{'Trigger': {'precision': 0.06319627304030788, 'recall': 0.1712403951701427, 'f1': 0.0923213493120284, 'number': 1822}, 'overall_precision': 0.06319627304030788, 'overall_recall': 0.1712403951701427, 'overall_f1': 0.0923213493120284, 'overall_accuracy': 0.8840432195427498}
